In [4]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

In [5]:
url='https://drive.google.com/file/d/1F5KnTS79MODxHypKVMB6hz1ZqUZyn0Cy/view?usp=sharing'
file_id=url.split('/')[-2]
dwn_url='https://drive.google.com/uc?id=' + file_id
df = pd.read_csv(dwn_url)
print(df.head())

   CommentId                              commentText   language  user_index  \
0     411618                        __steeve__joseph   Malayalam     26850.0   
1     411619                            aghana_shini   Malayalam     17103.0   
2     411620  ayyo pavam kuchu enthelum pattiyo entho  Malayalam     21720.0   
3     411621                                 kalakki   Malayalam     18615.0   
4     411622                             riya_ponnuz   Malayalam     24855.0   

   post_index  report_count_comment  report_count_post  like_count_comment  \
0    534771.0                   0.0                0.0                 0.0   
1    843407.0                   0.0                0.0                 0.0   
2    101538.0                   0.0               31.0                 0.0   
3    521985.0                   0.0                0.0                 0.0   
4   1051383.0                   0.0                0.0                 1.0   

   like_count_post  label                         

In [6]:
df=df.drop(columns=['CommentId','commentText','language','user_index','post_index','report_count_comment', 'report_count_post', 'like_count_comment','like_count_post','commentCleaned'])

In [7]:
df.head()

,label,translated_value
0,0.0,steevejoseph
1,0.0,Aghnashini
2,0.0,ayyo pavam kuchu enthelum patelum pattiyo entho
3,0.0,Brag
4,0.0,Reponguz


Spelling correction

In [8]:
import textblob

In [9]:
def preprocess_data(df):
   
    
    # Convert text to lowercase
    df['lower'] = df['translated_value'].str.strip().str.lower()
    return df

In [10]:
dff = preprocess_data(df)

In [11]:
dff['translated_value']=dff['lower']

In [12]:
dff.head()

,label,translated_value,lower
0,0.0,steevejoseph,steevejoseph
1,0.0,aghnashini,aghnashini
2,0.0,ayyo pavam kuchu enthelum patelum pattiyo entho,ayyo pavam kuchu enthelum patelum pattiyo entho
3,0.0,brag,brag
4,0.0,reponguz,reponguz


In [13]:
dff=dff.drop(columns=['lower'])

In [14]:
dff.head()

,label,translated_value
0,0.0,steevejoseph
1,0.0,aghnashini
2,0.0,ayyo pavam kuchu enthelum patelum pattiyo entho
3,0.0,brag
4,0.0,reponguz


In [15]:
X_temp, X_test, y_temp, y_test = train_test_split(dff, list(dff.label), test_size=0.2, random_state=0)

In [16]:
X_test.shape, X_temp.shape, len(y_test), len(y_temp)

((26153, 2), (104612, 2), 26153, 104612)

In [17]:
X_temp['label'] = y_temp

<ipython-input-17-696ff4fc08a7>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_temp['label'] = y_temp


In [18]:
X_temp.head()

,label,translated_value
29677,0.0,sup you
29254,0.0,kou gharar from
27234,1.0,baihu temple is eating
5709,0.0,nice
51176,0.0,want beautiful tanutu hi


In [19]:
zero = X_temp[X_temp['label'] == 0]
zero.shape[0]

72900

In [20]:
one = X_temp[X_temp['label'] == 1]
one.shape[0]

31712

In [21]:
ds = pd.concat([zero,one], axis = 0)
ds

,label,translated_value
29677,0.0,sup you
29254,0.0,kou gharar from
5709,0.0,nice
51176,0.0,want beautiful tanutu hi
9961,0.0,nicesuper
...,...,...
21243,1.0,thu muduka thu
45891,1.0,gand digi rindi
42613,1.0,nhi sewa to puri karo galiya bhi khao bahu hu ...
43567,1.0,hey matherchod randi


In [22]:
testdf = X_test
testdf['label'] = y_test
testdf.shape

<ipython-input-22-0eff1109cf1d>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  testdf['label'] = y_test


(26153, 2)

In [23]:
len(testdf[testdf['label']==0]),len(testdf[testdf['label']==1])

(18212, 7941)

In [24]:
ds = pd.concat([ds, testdf], axis = 0)

In [25]:
ds[ds['label']==0].shape, ds[ds['label']==1].shape

((91112, 2), (39653, 2))

In [26]:
corpus = []
for i in range(ds.shape[0]):
    corpus.append(ds.iloc[i][0])

# Creating TF-IDF

In [27]:
vectorizer1 = TfidfVectorizer(max_features=1000)
X1 = vectorizer1.fit_transform(ds['translated_value'])
feature_names1 = vectorizer1.get_feature_names()
denselist1 = X1.todense().tolist()
df2_train = pd.DataFrame(denselist1, columns=feature_names1)

In [28]:
tdf = df2_train
tdf['labelxyz'] = list(ds.label)

In [29]:
tdf_zero = tdf[tdf.labelxyz == 0]
tdf_zero.shape

tdf_one = tdf[tdf.labelxyz == 1]
tdf_one.shape

(39653, 1001)

In [30]:
X_train_zero = tdf_zero.sample(frac=0.9, random_state=0)
X_test_zero = tdf_zero.drop(X_train_zero.index)

In [31]:
X_train_one = tdf_one.sample(frac=0.9, random_state=0)
X_test_one = tdf_one.drop(X_train_one.index)

In [32]:
X_train_df = pd.concat([X_train_zero,X_train_one], axis = 0)
tfdf_train = X_train_df.drop(['labelxyz'], axis = 1)
y_train = list(X_train_df.labelxyz)
X_test_df = pd.concat([X_test_zero,X_test_one], axis = 0)
tfdf_test = X_test_df.drop(['labelxyz'], axis = 1)
y_test = list(X_test_df.labelxyz)

# Building ML models

In [33]:
model = ['LR','DT','GB','RF','KN']
accuracy = {'TF-IDF':[]}

In [37]:
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn import metrics
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.metrics import plot_confusion_matrix
from sklearn.metrics import precision_score, recall_score, f1_score

## Logistic Regression

In [ ]:
#TF-IDF
regressor_LR_tf = LogisticRegression(C= 1.0, penalty='l2', solver= 'liblinear')
regressor_LR_tf.fit(tfdf_train,y_train)
y_predict_LR_tf = regressor_LR_tf.predict(tfdf_test)
a=(regressor_LR_tf.score(tfdf_test,y_test))
accuracy['TF-IDF'].append(a)

print(metrics.confusion_matrix(y_test, y_predict_LR_tf))
print(metrics.classification_report(y_test, y_predict_LR_tf))
print(metrics.accuracy_score(y_test, y_predict_LR_tf))

[[8536  575]
 [1647 2318]]
              precision    recall  f1-score   support

         0.0       0.84      0.94      0.88      9111
         1.0       0.80      0.58      0.68      3965

    accuracy                           0.83     13076
   macro avg       0.82      0.76      0.78     13076
weighted avg       0.83      0.83      0.82     13076

0.830070357907617


## Decision Tree

In [ ]:
#TF-IDF
model_DT_tf = DecisionTreeClassifier() 
model_DT_tf.fit(tfdf_train,y_train)
y_predict_DT_tf = model_DT_tf.predict(tfdf_test)
a=(model_DT_tf.score(tfdf_test,y_test))
accuracy['TF-IDF'].append(a)

print(metrics.confusion_matrix(y_test, y_predict_DT_tf))
print(metrics.classification_report(y_test, y_predict_DT_tf))
print(metrics.accuracy_score(y_test, y_predict_DT_tf))

[[8204  907]
 [1745 2220]]
              precision    recall  f1-score   support

         0.0       0.82      0.90      0.86      9111
         1.0       0.71      0.56      0.63      3965

    accuracy                           0.80     13076
   macro avg       0.77      0.73      0.74     13076
weighted avg       0.79      0.80      0.79     13076

0.7971856836953197


## Gradient Boosting

In [ ]:
#TF-IDF
model_GB_tf = GradientBoostingClassifier() 
model_GB_tf.fit(tfdf_train,y_train)
y_predict_GB_tf = model_GB_tf.predict(tfdf_test)
a=(model_GB_tf.score(tfdf_test,y_test))
accuracy['TF-IDF'].append(a)
print(metrics.confusion_matrix(y_test, y_predict_GB_tf))
print(metrics.classification_report(y_test, y_predict_GB_tf))
print(metrics.accuracy_score(y_test, y_predict_GB_tf))

[[8945  166]
 [2843 1122]]
              precision    recall  f1-score   support

         0.0       0.76      0.98      0.86      9111
         1.0       0.87      0.28      0.43      3965

    accuracy                           0.77     13076
   macro avg       0.81      0.63      0.64     13076
weighted avg       0.79      0.77      0.73     13076

0.7698837565004588


## Random Forest

In [39]:
#TF-IDF
model_RF_tf = RandomForestClassifier()
model_RF_tf.fit(tfdf_train,y_train)
y_predict_RF_tf = model_RF_tf.predict(tfdf_test)
a=(model_RF_tf.score(tfdf_test,y_test))
accuracy['TF-IDF'].append(a)

print(metrics.confusion_matrix(y_test, y_predict_RF_tf))
print(metrics.classification_report(y_test, y_predict_RF_tf))
print(metrics.accuracy_score(y_test, y_predict_RF_tf))

[[8360  751]
 [1624 2341]]
              precision    recall  f1-score   support

         0.0       0.84      0.92      0.88      9111
         1.0       0.76      0.59      0.66      3965

    accuracy                           0.82     13076
   macro avg       0.80      0.75      0.77     13076
weighted avg       0.81      0.82      0.81     13076

0.8183695319669624


## K neighbors

In [38]:
#TF-IDF
model_KN_tf = KNeighborsClassifier() 
model_KN_tf.fit(tfdf_train,y_train)
y_predict_KN_tf = model_KN_tf.predict(tfdf_test)
a=(model_KN_tf.score(tfdf_test,y_test))
accuracy['TF-IDF'].append(a)
print(metrics.confusion_matrix(y_test, y_predict_KN_tf))
print(metrics.classification_report(y_test, y_predict_KN_tf))
print(metrics.accuracy_score(y_test, y_predict_KN_tf))

[[8695  416]
 [2610 1355]]
              precision    recall  f1-score   support

         0.0       0.77      0.95      0.85      9111
         1.0       0.77      0.34      0.47      3965

    accuracy                           0.77     13076
   macro avg       0.77      0.65      0.66     13076
weighted avg       0.77      0.77      0.74     13076

0.768583664729275
